In [ ]:
# from __future__ import print_function
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials

from datetime import datetime

# If modifying these scopes, delete the file token.json.
SCOPES = ['https://www.googleapis.com/auth/spreadsheets', 'https://www.googleapis.com/auth/drive']

# The ID and range of a sample spreadsheet.
spreadsheet_id = ''
range_name = 'A1:N100'
value_input_option = 'USER_ENTERED'

def createCredential():
    creds = None
    # The file token.json stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('token.json', 'w') as token:
            token.write(creds.to_json())
    return creds
            
def createSheetAndUpdate(values):
    """Shows basic usage of the Sheets API.
    Prints values from a sample spreadsheet.
    """
    creds = createCredential()

    service = build('sheets', 'v4', credentials=creds)

    title = "제목"
    body = {
        'properties': {
            'title': title
        },
    }
    request = service.spreadsheets().create(body=body)
    response = request.execute()

    # TODO: Change code below to process the `response` dict:
    spreadsheet_id = response.get('spreadsheetId')
    
    body = {
        'values': values
    }
    result = service.spreadsheets().values().update(
        spreadsheetId=spreadsheet_id, range=range_name,
        valueInputOption=value_input_option, body=body).execute()
    print('{0} cells updated.'.format(result.get('updatedCells')))
    return spreadsheet_id
    
def shareSheet(file_id):
    creds = createCredential()
    drive_service = build('drive', 'v3', credentials=creds)

    batch = drive_service.new_batch_http_request(callback=callback)
    user_permission = {
        'emailAddress': 'someone@gmail.com',
        'type': 'user',
        'role': 'writer'
    }
    batch.add(drive_service.permissions().create(
            fileId=file_id,
            body=user_permission,
            fields='id',
    ))
    batch.execute()
    
def callback(request_id, response, exception):
    if exception:
        # Handle error
        print ("exception: %s" % exception)
    else:
        print ("Permission Id: %s" % response.get('id'))

if __name__ == '__main__':
    values = ['test']
    file_id = createSheetAndUpdate(a)
    shareSheet(file_id)